In [1]:
import os

PRESET = os.environ.setdefault("PRESET", "test_binary_classification_eval")
SELF_PARTY = os.environ.setdefault("SELF_PARTY", "sim")


In [2]:
import ray
from opentelemetry import trace
from opentelemetry.sdk.environment_variables import OTEL_SERVICE_NAME

from secretnote.instrumentation import Profiler
from secretnote.instrumentation.sdk import (
    setup_tracing,
    setup_debug_exporter,
    setup_memory_exporter,
    inherit_tracing_context,
    setup_jsonlines_exporter,
)
from secretnote.instrumentation.rules import create_default_rules

ray.shutdown()

create_default_rules()

os.environ[OTEL_SERVICE_NAME] = f"{PRESET}:{SELF_PARTY}"

setup_tracing()
setup_debug_exporter()
setup_jsonlines_exporter(f"{PRESET}.{SELF_PARTY}")

exporter = setup_memory_exporter()

with trace.get_tracer(__name__).start_as_current_span(
    "sim_trace",
    context=inherit_tracing_context(),
):
    with Profiler():
        with open(f"presets/{PRESET}/_world.py") as f:
            _world = compile(f.read(), f.name, "exec")
        with open(f"presets/{PRESET}/_algorithm.py") as f:
            _algorithm = compile(f.read(), f.name, "exec")
        exec(_world, globals())
        exec(_algorithm, globals())


2023-10-19 18:55:27,473	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-10-19 18:55:30,847	INFO worker.py:1612 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
INFO:jax._src.xla_bridge:Unable to initialize backend 'plugin': xla_extension has no attributes named get_plugin_device_client. Compile TensorFlow with //tensorflow/compiler/xla/python:enable_plugin_device set to true (defaults to false) to enable this.
(_run pid=26611) INFO:jax._src.xla_bridge:Unable to initialize backen